In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')
graphlab.canvas.set_target('ipynb')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1537883971.log


This non-commercial license of GraphLab Create for academic use is assigned to r38411@yahoo.com and will expire on August 31, 2019.


In [3]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


In [7]:
image_train['label'].show(view='Categorical')
image_test['label'].show(view='Categorical')
image_test.show()

In [ ]:
image_train['label'].sketch_summary()

# Question 1: there are 4 unique labels: automobile, cat, dog and bird in image_train

## Question 2: split the data into 4 groups based on the label

In [ ]:
data_list = {}
data_model = {}
i_label = image_train['label'].unique()
for i in range(len(i_label)):
    data_list[i_label[i]] = image_train.filter_by(i_label[i], 'label')
    data_model[i_label[i]] = graphlab.nearest_neighbors.create(data_list[i_label[i]],
                                                                features = ['deep_features'],
                                                                label = 'id')

In [ ]:
the_cat = image_test[0:1]
the_cat['image'].show()

In [ ]:
q2_nearest = {}
q2_nearest_1 = {}
for i in range(len(i_label)):
    q2_nearest[i_label[i]] = data_model[i_label[i]].query(the_cat)
    q2_nearest_1[i_label[i]] = q2_nearest[i_label[i]][0]['reference_label']
q2_nearest_1

# Question 2 answer: nearest neighbor of the the_cat: 
# automobile model: 49314, bird_model: 49573, cat_model: 16289, dog_model: 16976

## Question 3

In [ ]:
q2_nearest['automobile']

In [ ]:
for i in q2_nearest.keys():
    print i, ': ', 'avg distance = ', sum(q2_nearest[i]['distance'])/len(q2_nearest[i]['distance'])

# Question 3: avg distance from each model: 
# automobile: 40.7548, cat: 36.155, bird: 39.2939, dog: 37.77

## Question 4

In [ ]:
## split the image test data based on the label

In [ ]:
test_data_group = {}
for i in range(len(i_label)):
    test_data_group[i_label[i]] = image_test.filter_by(i_label[i], 'label')
    print test_data_group[i_label[i]].head()

In [ ]:
def test_group(which_test_group):
    neighbors = {}
    for i in range(len(i_label)):
        neighbors[i_label[i]] = data_model[i_label[i]].query(test_data_group[which_test_group], k=1)
        print "***** inside test_group(): " + "data_model = " + i_label[i] + ", test_group = " + which_test_group
    return neighbors

In [ ]:
def build_distance(which_test_group):
    group_distance = graphlab.SFrame()
    for i in range(len(i_label)):
        col_name = which_test_group + '-' + i_label[i]
        group_distance[col_name] = neighbors[i_label[i]]['distance']
    print "******** current test group is: " + which_test_group
    print group_distance.head()
    return group_distance

In [ ]:
def is_group_correct(row):
    same_group_name = current_test_group_name + '-' + current_test_group_name
    if min(row, key=row.get) == same_group_name:
        return 1
    else:
        return 0

In [ ]:
def group_match_analysis(test_group_name):
    neighbors_1 = test_group(test_group_name)
    group_distance_1 = build_distance(test_group_name)
    current_test_group_name = test_group_name
    ans = group_distance.apply(is_group_correct).sum()
    print "*********" + "test_group_name = " + test_group_name + " matches " + str(ans)
    return ans

In [ ]:
num_match = {}
current_test_group_name = ""
for i in i_label:
    num_match[i] = group_match_analysis(i)
num_match

# Question 4: test_imag_group = dog, train_image_group = dog, accuracy = 67.8%, # matches=678

In [ ]:
cat_neighbors = {}
test_group_name = 'cat'
for i in range(len(i_label)):
    cat_neighbors[i_label[i]] = data_model[i_label[i]].query(test_data_group[test_group_name], k=1)
    print i_label[i]
    print cat_neighbors[i_label[i]].head()

In [ ]:
## build the SFrame

In [ ]:
cat_distance = graphlab.SFrame()
for i in range(len(i_label)):
    col_name = test_group_name + '-' + i_label[i]
    cat_distance[col_name] = cat_neighbors[i_label[i]]['distance']
cat_distance.head()

In [ ]:
## now compute how many of them dog-dog is min distance

In [ ]:
def is_cat_correct(row):
    if min(row, key=row.get) == "cat-cat":
        return 1
    else:
        return 0
cat_distance.apply(is_cat_correct).sum()